In [54]:
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

# Parameters Config

In [77]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [78]:
TRAINING_SIZE = 80000
DIGITS = 3
REVERSE = False
ADD_SUB_MIX = True
MAXLEN = DIGITS + 1 + DIGITS
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1
chars = '0123456789+- ' if ADD_SUB_MIX else '0123456789+'
op_option = list('+-' if ADD_SUB_MIX else '+')

In [79]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [80]:
ctable = CharacterTable(chars)

In [81]:
ctable.indices_char

{0: ' ',
 1: '+',
 2: '-',
 3: '0',
 4: '1',
 5: '2',
 6: '3',
 7: '4',
 8: '5',
 9: '6',
 10: '7',
 11: '8',
 12: '9'}

# Data Generation

In [82]:
questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b, op = f(), f(), np.random.choice(op_option)
    #key = tuple(sorted((a, b)))
    key = a, b, op
    if key in seen:
        continue
    seen.add(key)
    q = '{}{}{}'.format(a, op, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a+b if op is '+' else a-b)
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 80000


In [83]:
print(questions[:10], expected[:10])

['11-63  ', '12-1   ', '5-1    ', '7-5    ', '27+312 ', '26-17  ', '3+16   ', '22+61  ', '58+49  ', '634-269'] ['-52 ', '11  ', '4   ', '2   ', '-285', '9   ', '-13 ', '-39 ', '9   ', '365 ']


# Processing

In [71]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

Vectorization...


In [72]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
split_at = len(x) // 2
train_x, test_x = x[:split_at], x[split_at:]
train_y, test_y = y[:split_at], y[split_at:]

split_at = len(train_x) - len(train_x) // 10
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)

Training Data:
(36000, 7, 13)
(36000, 4, 13)
Validation Data:
(4000, 7, 13)
(4000, 4, 13)
Testing Data:
(40000, 7, 13)
(40000, 4, 13)


In [73]:
print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

input:  [[[False False False False False False False  True False False False
   False False]
  [False False False False False False False False False False False
    True False]
  [False False False False False False False False False False  True
   False False]
  [False  True False False False False False False False False False
   False False]
  [False False False False False False False  True False False False
   False False]
  [False False False False False False False  True False False False
   False False]
  [ True False False False False False False False False False False
   False False]]

 [[False False False False False False  True False False False False
   False False]
  [False False False False False False False False False False False
    True False]
  [False False False False False False False False False False False
   False  True]
  [False False  True False False False False False False False False
   False False]
  [False False False False False False False False Fals

# Build Model

In [74]:
print('Build model...')
model = Sequential()
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 1))
for _ in range(LAYERS):
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 128)               72704     
_________________________________________________________________
repeat_vector_3 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_3 (TimeDist (None, 4, 13)             1677      
_________________________________________________________________
activation_3 (Activation)    (None, 4, 13)             0         
Total params: 205,965
Trainable params: 205,965
Non-trainable params: 0
_________________________________________________________________


# Training

In [75]:
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    """
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)
    """


--------------------------------------------------
Iteration 0
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 18s 494us/step - loss: 2.0303 - acc: 0.3108 - val_loss: 1.8593 - val_acc: 0.3531

--------------------------------------------------
Iteration 1
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 16s 450us/step - loss: 1.7629 - acc: 0.3732 - val_loss: 1.6748 - val_acc: 0.3935

--------------------------------------------------
Iteration 2
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 16s 431us/step - loss: 1.6147 - acc: 0.4182 - val_loss: 1.5528 - val_acc: 0.4396

--------------------------------------------------
Iteration 3
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 16s 442us/step - loss: 1.5062 - acc: 0.4606 - val_loss: 1.4584 - val_acc: 0.4

36000/36000 [==============================] - 15s 424us/step - loss: 0.8337 - acc: 0.6969 - val_loss: 0.8429 - val_acc: 0.6901

--------------------------------------------------
Iteration 33
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 15s 416us/step - loss: 0.8193 - acc: 0.7024 - val_loss: 0.8548 - val_acc: 0.6844

--------------------------------------------------
Iteration 34
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 15s 421us/step - loss: 0.8062 - acc: 0.7072 - val_loss: 0.8160 - val_acc: 0.6974

--------------------------------------------------
Iteration 35
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 15s 418us/step - loss: 0.7902 - acc: 0.7134 - val_loss: 0.8241 - val_acc: 0.6943

--------------------------------------------------
Iteration 36
Train on 36000 samples, validate on 4000 samples
Ep

36000/36000 [==============================] - 15s 430us/step - loss: 0.0656 - acc: 0.9856 - val_loss: 0.0923 - val_acc: 0.9742

--------------------------------------------------
Iteration 66
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 15s 422us/step - loss: 0.0637 - acc: 0.9857 - val_loss: 0.0853 - val_acc: 0.9758

--------------------------------------------------
Iteration 67
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 13s 371us/step - loss: 0.0731 - acc: 0.9808 - val_loss: 0.1402 - val_acc: 0.9575

--------------------------------------------------
Iteration 68
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 14s 390us/step - loss: 0.0668 - acc: 0.9831 - val_loss: 0.1192 - val_acc: 0.9626

--------------------------------------------------
Iteration 69
Train on 36000 samples, validate on 4000 samples
Ep

36000/36000 [==============================] - 16s 431us/step - loss: 0.0126 - acc: 0.9984 - val_loss: 0.0368 - val_acc: 0.9894

--------------------------------------------------
Iteration 99
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 16s 448us/step - loss: 0.0119 - acc: 0.9986 - val_loss: 0.0338 - val_acc: 0.9904


# Testing

In [76]:
right = 0
preds = model.predict_classes(test_x, verbose=0)
for i in range(len(preds)):
    q = ctable.decode(test_x[i])
    correct = ctable.decode(test_y[i])
    guess = ctable.decode(preds[i], calc_argmax=False)
    print('Q', q[::-1] if REVERSE else q, end=' ')
    print('T', correct, end=' ')
    if correct == guess:
        print(colors.ok + '☑' + colors.close, end=' ')
        right += 1
    else:
        print(colors.fail + '☒' + colors.close, end=' ')
    print(guess)
print("MSG : Accuracy is {}".format(right / len(preds)))

Q 129+613 T -484 ☑ -484
Q 224-423 T -199 ☑ -199
Q 446-942 T -496 ☑ -496
Q 986-610 T 376  ☑ 376 
Q 595+727 T -132 ☑ -132
Q 176+46  T 130  ☑ 130 
Q 517+406 T 111  ☑ 111 
Q 684+250 T 434  ☑ 434 
Q 493-652 T -159 ☑ -159
Q 906+219 T 687  ☑ 687 
Q 354+48  T 306  ☑ 306 
Q 348-813 T -465 ☑ -465
Q 232+273 T -41  ☑ -41 
Q 429+659 T -230 ☑ -230
Q 291-436 T -145 ☑ -145
Q 905-240 T 665  ☑ 665 
Q 6-631   T -625 ☑ -625
Q 724-634 T 90   ☑ 90  
Q 774+886 T -112 ☑ -112
Q 856-3   T 853  ☑ 853 
Q 90-397  T -307 ☑ -307
Q 97-949  T -852 ☑ -852
Q 908+740 T 168  ☑ 168 
Q 821-140 T 681  ☑ 681 
Q 831+996 T -165 ☑ -165
Q 744-367 T 377  ☑ 377 
Q 505+96  T 409  ☑ 409 
Q 741-916 T -175 ☑ -175
Q 923+149 T 774  ☑ 774 
Q 673-337 T 336  ☑ 336 
Q 837+716 T 121  ☑ 121 
Q 451+996 T -545 ☑ -545
Q 892+869 T 23   ☑ 23  
Q 616-58  T 558  ☑ 558 
Q 549-935 T -386 ☑ -386
Q 784-585 T 199  ☑ 199 
Q 785-953 T -168 ☑ -168
Q 358+969 T -611 ☑ -611
Q 912-723 T 189  ☑ 189 
Q 936+887 T 49   ☑ 49  
Q 920-853 T 67   ☑ 67  
Q 64-948  T -884

Q 398+829 T -431 ☑ -431
Q 613+574 T 39   ☒ 49  
Q 453-969 T -516 ☑ -516
Q 679+342 T 337  ☑ 337 
Q 606+817 T -211 ☑ -211
Q 418-709 T -291 ☑ -291
Q 204+976 T -772 ☑ -772
Q 202+459 T -257 ☑ -257
Q 961-512 T 449  ☑ 449 
Q 303-237 T 66   ☑ 66  
Q 143+43  T 100  ☑ 100 
Q 415+365 T 50   ☑ 50  
Q 373+812 T -439 ☑ -439
Q 944+989 T -45  ☑ -45 
Q 975-403 T 572  ☑ 572 
Q 642+212 T 430  ☑ 430 
Q 312-357 T -45  ☑ -45 
Q 2+98    T -96  ☒ -6  
Q 911+397 T 514  ☑ 514 
Q 808+461 T 347  ☑ 347 
Q 850-827 T 23   ☑ 23  
Q 674+230 T 444  ☑ 444 
Q 454-236 T 218  ☑ 218 
Q 273+515 T -242 ☑ -242
Q 786+449 T 337  ☑ 337 
Q 39-974  T -935 ☑ -935
Q 87-623  T -536 ☑ -536
Q 722+557 T 165  ☑ 165 
Q 203-822 T -619 ☑ -619
Q 547-965 T -418 ☑ -418
Q 637-137 T 500  ☑ 500 
Q 233+679 T -446 ☑ -446
Q 428-310 T 118  ☑ 118 
Q 560-368 T 192  ☑ 192 
Q 993+425 T 568  ☑ 568 
Q 869-939 T -70  ☑ -70 
Q 519-822 T -303 ☑ -303
Q 176+172 T 4    ☑ 4   
Q 733-988 T -255 ☑ -255
Q 790+947 T -157 ☑ -157
Q 420-752 T -332 ☑ -332
Q 682-992 T -310

Q 403-674 T -271 ☑ -271
Q 728+125 T 603  ☑ 603 
Q 353+828 T -475 ☑ -475
Q 710+150 T 560  ☑ 560 
Q 255+349 T -94  ☑ -94 
Q 197+922 T -725 ☑ -725
Q 127-606 T -479 ☑ -479
Q 90+219  T -129 ☑ -129
Q 936+854 T 82   ☑ 82  
Q 502-45  T 457  ☑ 457 
Q 481-151 T 330  ☑ 330 
Q 147-436 T -289 ☑ -289
Q 109-538 T -429 ☑ -429
Q 968+94  T 874  ☑ 874 
Q 598-963 T -365 ☑ -365
Q 819-70  T 749  ☑ 749 
Q 105-782 T -677 ☑ -677
Q 205+101 T 104  ☑ 104 
Q 937+690 T 247  ☑ 247 
Q 809+385 T 424  ☑ 424 
Q 520-296 T 224  ☑ 224 
Q 862+589 T 273  ☑ 273 
Q 765-110 T 655  ☑ 655 
Q 390-955 T -565 ☑ -565
Q 404+69  T 335  ☑ 335 
Q 530+251 T 279  ☑ 279 
Q 861+375 T 486  ☑ 486 
Q 913-480 T 433  ☑ 433 
Q 828-576 T 252  ☑ 252 
Q 223-293 T -70  ☑ -70 
Q 401+649 T -248 ☑ -248
Q 705-315 T 390  ☑ 390 
Q 128+175 T -47  ☑ -47 
Q 736-60  T 676  ☑ 676 
Q 584-668 T -84  ☑ -84 
Q 94+8    T 86   ☑ 86  
Q 203-697 T -494 ☑ -494
Q 442+501 T -59  ☑ -59 
Q 821+430 T 391  ☑ 391 
Q 78-746  T -668 ☑ -668
Q 576+555 T 21   ☑ 21  
Q 65-228  T -163

Q 587+668 T -81  ☑ -81 
Q 535-134 T 401  ☑ 401 
Q 610-734 T -124 ☑ -124
Q 529+227 T 302  ☑ 302 
Q 500+293 T 207  ☑ 207 
Q 353+151 T 202  ☑ 202 
Q 581-667 T -86  ☑ -86 
Q 325+36  T 289  ☑ 289 
Q 512+141 T 371  ☑ 371 
Q 323+970 T -647 ☑ -647
Q 648-435 T 213  ☑ 213 
Q 855+938 T -83  ☑ -83 
Q 402+592 T -190 ☑ -190
Q 210+185 T 25   ☑ 25  
Q 693+948 T -255 ☑ -255
Q 158-622 T -464 ☑ -464
Q 253-890 T -637 ☑ -637
Q 768+277 T 491  ☑ 491 
Q 943-916 T 27   ☑ 27  
Q 349+12  T 337  ☒ 327 
Q 564-470 T 94   ☑ 94  
Q 4+581   T -577 ☑ -577
Q 364+125 T 239  ☑ 239 
Q 998-478 T 520  ☑ 520 
Q 764+53  T 711  ☑ 711 
Q 902-404 T 498  ☑ 498 
Q 148-727 T -579 ☑ -579
Q 254+102 T 152  ☑ 152 
Q 719-64  T 655  ☑ 655 
Q 327+403 T -76  ☑ -76 
Q 244-485 T -241 ☑ -241
Q 32+711  T -679 ☑ -679
Q 90+966  T -876 ☑ -876
Q 485+47  T 438  ☑ 438 
Q 269-661 T -392 ☑ -392
Q 23-839  T -816 ☑ -816
Q 850+147 T 703  ☑ 703 
Q 853-700 T 153  ☑ 153 
Q 140+464 T -324 ☑ -324
Q 210+8   T 202  ☒ 211 
Q 134+138 T -4   ☑ -4  
Q 967-418 T 549 

Q 952+892 T 60   ☑ 60  
Q 554-798 T -244 ☑ -244
Q 186-3   T 183  ☑ 183 
Q 501+957 T -456 ☑ -456
Q 439+666 T -227 ☑ -227
Q 899-64  T 835  ☑ 835 
Q 314+731 T -417 ☑ -417
Q 670-5   T 665  ☑ 665 
Q 743+475 T 268  ☑ 268 
Q 173+1   T 172  ☑ 172 
Q 805+995 T -190 ☑ -190
Q 0+275   T -275 ☑ -275
Q 46-96   T -50  ☒ -40 
Q 941-227 T 714  ☑ 714 
Q 847-435 T 412  ☑ 412 
Q 501+376 T 125  ☑ 125 
Q 157+701 T -544 ☑ -544
Q 117-686 T -569 ☑ -569
Q 658+67  T 591  ☑ 591 
Q 527-199 T 328  ☑ 328 
Q 129-857 T -728 ☑ -728
Q 218+511 T -293 ☑ -293
Q 748+300 T 448  ☑ 448 
Q 507+880 T -373 ☑ -373
Q 79+624  T -545 ☑ -545
Q 723-259 T 464  ☑ 464 
Q 774-428 T 346  ☑ 346 
Q 346-68  T 278  ☑ 278 
Q 412+640 T -228 ☑ -228
Q 594-660 T -66  ☑ -66 
Q 94-794  T -700 ☑ -700
Q 618+912 T -294 ☑ -294
Q 398+139 T 259  ☑ 259 
Q 160+105 T 55   ☑ 55  
Q 177-446 T -269 ☑ -269
Q 68+854  T -786 ☑ -786
Q 314-624 T -310 ☑ -310
Q 458+281 T 177  ☑ 177 
Q 464+331 T 133  ☑ 133 
Q 6-24    T -18  ☑ -18 
Q 853-336 T 517  ☑ 517 
Q 467+170 T 297 

...

Q 75-618  T -543 ☑ -543
Q 307-350 T -43  ☑ -43 
Q 913+408 T 505  ☑ 505 
Q 414-511 T -97  ☑ -97 
Q 99-994  T -895 ☑ -895
Q 3+676   T -673 ☑ -673
Q 42-579  T -537 ☑ -537
Q 358+579 T -221 ☑ -221
Q 756-799 T -43  ☑ -43 
Q 119+882 T -763 ☑ -763
Q 551-946 T -395 ☑ -395
Q 379+591 T -212 ☑ -212
Q 632-869 T -237 ☑ -237
Q 257+853 T -596 ☑ -596
Q 317+980 T -663 ☑ -663
Q 512-259 T 253  ☑ 253 
Q 592+783 T -191 ☑ -191
Q 929-51  T 878  ☑ 878 
Q 843+230 T 613  ☑ 613 
Q 389-399 T -10  ☑ -10 
Q 730-766 T -36  ☑ -36 
Q 352+740 T -388 ☑ -388
Q 49+204  T -155 ☑ -155
Q 628+863 T -235 ☑ -235
Q 762-729 T 33   ☑ 33  
Q 329-17  T 312  ☑ 312 
Q 516-112 T 404  ☑ 404 
Q 453-989 T -536 ☑ -536
Q 237+299 T -62  ☑ -62 
Q 111-119 T -8   ☑ -8  
Q 76+417  T -341 ☑ -341
Q 300+897 T -597 ☑ -597
Q 163-181 T -18  ☑ -18 
Q 842-290 T 552  ☑ 552 
Q 653-931 T -278 ☑ -278
Q 520-232 T 288  ☑ 288 
Q 734-168 T 566  ☑ 566 
Q 316-59  T 257  ☒ 247 
Q 736-233 T 503  ☑ 503 
Q 640+471 T 169  ☑ 169 
Q 247-178 T 69   ☑ 69  
Q 455+991 T -536

Q 423-177 T 246  ☑ 246 
Q 988+767 T 221  ☑ 221 
Q 59-108  T -49  ☑ -49 
Q 801-492 T 309  ☑ 309 
Q 761+490 T 271  ☑ 271 
Q 689-848 T -159 ☑ -159
Q 729-55  T 674  ☑ 674 
Q 580-812 T -232 ☑ -232
Q 732-137 T 595  ☑ 595 
Q 514+129 T 385  ☑ 385 
Q 650-802 T -152 ☑ -152
Q 500-735 T -235 ☑ -235
Q 378+117 T 261  ☑ 261 
Q 870-513 T 357  ☑ 357 
Q 960+383 T 577  ☑ 577 
Q 192-624 T -432 ☑ -432
Q 769+873 T -104 ☑ -104
Q 380-39  T 341  ☑ 341 
Q 977+235 T 742  ☑ 742 
Q 980-422 T 558  ☑ 558 
Q 927+5   T 922  ☑ 922 
Q 685+430 T 255  ☑ 255 
Q 569-141 T 428  ☑ 428 
Q 322+7   T 315  ☑ 315 
Q 26-974  T -948 ☑ -948
Q 649+910 T -261 ☑ -261
Q 314-577 T -263 ☑ -263
Q 923-522 T 401  ☑ 401 
Q 76-382  T -306 ☑ -306
Q 810-511 T 299  ☑ 299 
Q 815-416 T 399  ☑ 399 
Q 472-179 T 293  ☑ 293 
Q 833-877 T -44  ☑ -44 
Q 794+251 T 543  ☑ 543 
Q 315+733 T -418 ☑ -418
Q 710-71  T 639  ☑ 639 
Q 163+98  T 65   ☑ 65  
Q 54-98   T -44  ☑ -44 
Q 618-649 T -31  ☑ -31 
Q 158-839 T -681 ☑ -681
Q 276+279 T -3   ☑ -3  
Q 737+896 T -159

Q 106+720 T -614 ☑ -614
Q 404+776 T -372 ☑ -372
Q 84+148  T -64  ☑ -64 
Q 189-951 T -762 ☑ -762
Q 48+491  T -443 ☑ -443
Q 107+538 T -431 ☑ -431
Q 248-951 T -703 ☑ -703
Q 107-290 T -183 ☑ -183
Q 4-823   T -819 ☑ -819
Q 662+262 T 400  ☑ 400 
Q 956-775 T 181  ☑ 181 
Q 735+812 T -77  ☑ -77 
Q 381+546 T -165 ☑ -165
Q 455-377 T 78   ☑ 78  
Q 361+454 T -93  ☑ -93 
Q 477-885 T -408 ☑ -408
Q 878+686 T 192  ☑ 192 
Q 220+188 T 32   ☑ 32  
Q 318-565 T -247 ☑ -247
Q 467+889 T -422 ☑ -422
Q 161-498 T -337 ☑ -337
Q 684+374 T 310  ☑ 310 
Q 849+21  T 828  ☑ 828 
Q 872-884 T -12  ☑ -12 
Q 875-352 T 523  ☑ 523 
Q 55+829  T -774 ☑ -774
Q 286-444 T -158 ☑ -158
Q 839+405 T 434  ☑ 434 
Q 168+859 T -691 ☑ -691
Q 488-170 T 318  ☑ 318 
Q 854+927 T -73  ☑ -73 
Q 455+134 T 321  ☑ 321 
Q 361-253 T 108  ☑ 108 
Q 137-652 T -515 ☑ -515
Q 957+431 T 526  ☑ 526 
Q 709+292 T 417  ☑ 417 
Q 441+938 T -497 ☑ -497
Q 249-946 T -697 ☑ -697
Q 388+12  T 376  ☑ 376 
Q 881+828 T 53   ☑ 53  
Q 371+409 T -38  ☑ -38 
Q 390+685 T -295

Q 276+600 T -324 ☑ -324
Q 298-508 T -210 ☑ -210
Q 30-749  T -719 ☑ -719
Q 924+44  T 880  ☑ 880 
Q 344-972 T -628 ☑ -628
Q 964+559 T 405  ☑ 405 
Q 903-131 T 772  ☑ 772 
Q 321-910 T -589 ☑ -589
Q 410-745 T -335 ☑ -335
Q 82+561  T -479 ☑ -479
Q 316-648 T -332 ☑ -332
Q 68+398  T -330 ☑ -330
Q 982-320 T 662  ☑ 662 
Q 726+621 T 105  ☑ 105 
Q 350-126 T 224  ☑ 224 
Q 820-371 T 449  ☑ 449 
Q 270-284 T -14  ☑ -14 
Q 914-518 T 396  ☑ 396 
Q 332-891 T -559 ☑ -559
Q 357+636 T -279 ☑ -279
Q 437-77  T 360  ☑ 360 
Q 403+675 T -272 ☑ -272
Q 613-69  T 544  ☑ 544 
Q 770+947 T -177 ☑ -177
Q 500+100 T 400  ☑ 400 
Q 779-700 T 79   ☑ 79  
Q 29-535  T -506 ☑ -506
Q 929-839 T 90   ☑ 90  
Q 555+749 T -194 ☑ -194
Q 210+320 T -110 ☑ -110
Q 960-674 T 286  ☑ 286 
Q 900+170 T 730  ☑ 730 
Q 763+898 T -135 ☑ -135
Q 497+576 T -79  ☑ -79 
Q 602-85  T 517  ☑ 517 
Q 963-421 T 542  ☑ 542 
Q 469-188 T 281  ☑ 281 
Q 923-252 T 671  ☑ 671 
Q 546-568 T -22  ☑ -22 
Q 188+253 T -65  ☑ -65 
Q 798+75  T 723  ☑ 723 
Q 606-916 T -310

Q 432+290 T 142  ☑ 142 
Q 746-101 T 645  ☑ 645 
Q 586+816 T -230 ☑ -230
Q 71-717  T -646 ☑ -646
Q 491+170 T 321  ☒ 331 
Q 873+182 T 691  ☑ 691 
Q 891+418 T 473  ☑ 473 
Q 243+588 T -345 ☑ -345
Q 119+279 T -160 ☑ -160
Q 540-5   T 535  ☑ 535 
Q 350+950 T -600 ☑ -600
Q 302+244 T 58   ☑ 58  
Q 313+449 T -136 ☑ -136
Q 127+4   T 123  ☑ 123 
Q 292-482 T -190 ☑ -190
Q 329+503 T -174 ☑ -174
Q 636+293 T 343  ☑ 343 
Q 928-25  T 903  ☑ 903 
Q 992-323 T 669  ☑ 669 
Q 999-263 T 736  ☑ 736 
Q 66+657  T -591 ☑ -591
Q 967+248 T 719  ☑ 719 
Q 163-974 T -811 ☑ -811
Q 244+838 T -594 ☑ -594
Q 702+889 T -187 ☑ -187
Q 834+261 T 573  ☑ 573 
Q 313-571 T -258 ☑ -258
Q 831+547 T 284  ☑ 284 
Q 155+23  T 132  ☑ 132 
Q 21+681  T -660 ☑ -660
Q 488+930 T -442 ☑ -442
Q 656-45  T 611  ☑ 611 
Q 975+84  T 891  ☑ 891 
Q 510-832 T -322 ☑ -322
Q 123+250 T -127 ☑ -127
Q 474-910 T -436 ☑ -436
Q 385-986 T -601 ☑ -601
Q 85+239  T -154 ☑ -154
Q 451+368 T 83   ☑ 83  
Q 705-380 T 325  ☑ 325 
Q 414+783 T -369 ☑ -369
Q 757-406 T 351 

In [ ]:
test_x